# 메일링 리스트 데이터베이스 연동
## 메일링 리스트 구현

### 메일 발송 준비

In [14]:
# 모듈 참조
from mylibrary import MyMailer
import datetime as dt

In [15]:
# 날짜 성분값 초기화
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day
print(year, month, day)

2025 10 27


### 메일 발송 준비(2)

In [16]:
#메일 발신자 정보
fromAddr = "운영지원팀 <rnjswodyd1@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

### 메일 발송 준비(3)

In [5]:
# 메일 본문 가져오기
with open('mail/content.txt', 'r', encoding = 'utf - 8') as f:
    contentTmpl = f.read()
    print(contentTmpl)

안녕하세요 {name}님

{yy}년도 {mm}월 급여명세서와 결산보고서 보내드립니다.

귀하의 노고에 감사드립니다.

- {yy}년 {mm}월 {dd}일 / 운영지원팀 드림


### 수신자 목록CSV에 대한 반복 처리


In [17]:
with open("mail/mail_list.csv", "r", encoding = "euc-kr") as f:
    csv = f.readlines()

    # CSV의 라인 수 만큼 반복
    for line in csv:
        name, email, file1, file2 = line.strip().split(",")
        #print(name,email, file1, file2)

        toAddr = "{name} <{email}>".format(name=name, email=email)
        #print(toAddr)

        # 메일 제목
        subject = subjectTmpl.format(name=name, yy=year, mm=month)
        #print(subject)

        # 메일 내용
        content = contentTmpl.format(name=name, yy=year, mm=month, dd=day)
        #print(content)

        # 메일 보내기
        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

document.pptx 가 첨부되었습니다
pay1.xlsx 가 첨부되었습니다
document.pptx 가 첨부되었습니다
pay2.xlsx 가 첨부되었습니다
document.pptx 가 첨부되었습니다
pay3.xlsx 가 첨부되었습니다


### 필요한 패키지 참조하기

In [23]:
# 프로그램에 딜레이를 적용하는 기능을 제공하는 모듈
import time

# 날짜 처리 모듈
import datetime as dt

# 메일 발송 모듈
from mylibrary import MyMailer

# 비동기 처리 기능을 제공하는 모듈
import concurrent.futures as futures

### 주어진 시간 동안 딜레이가 발생하는 함수 정의


In [21]:
# 다소 시간이 오래 걸리는 상황을 가정한 함수

def timeWork(name, seconds):
    print("[%s] 작업을 %d초 동안 수행" % (name, seconds))

    for i in range(0, seconds):
        time.sleep(1)
        print("[%s] %d초]" % (name, i+1))

    print("[%s] 작업이 종료되었음" % name)


### 동기식 함수 호출

In [ ]:
# 순차적으로 작업이 수행된다

startTime = dt.datetime.now()

timeWork("A", 3)
timeWork("B", 5)
timeWork("C", 2)

endtime = dt.datetime.now()
workTime = endtime - startTime
print("작업에 소요된 시간은 총 %s초 입니다" % workTime.seconds)

[A] 작업을 3초 동안 수행
[A] 1초]
[A] 2초]
[A] 3초]
[A] 작업이 종료되었음
[B] 작업을 5초 동안 수행
[B] 1초]
[B] 2초]
[B] 3초]
[B] 4초]
[B] 5초]
[B] 작업이 종료되었음
[C] 작업을 2초 동안 수행
[C] 1초]
[C] 2초]
[C] 작업이 종료되었음
작업에 소요된 시간은 총 10초 입니다


### 비동기식 함수 호출


In [28]:
# ThreadPollExecutor에 의한 비동기식 함수 호출

starTime = dt.datetime.now()

with futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.submit(timeWork, "A", 3)
    executor.submit(timeWork, "B", 5)
    executor.submit(timeWork, "C", 2)

endtime = dt.datetime.now()
workTime = endtime - startTime
print("작업에 소요된 시간은 총 %s초 입니다." % workTime.seconds)

[A] 작업을 3초 동안 수행
[B] 작업을 5초 동안 수행
[C] 작업을 2초 동안 수행
[A] 1초]
[B] 1초]
[C] 1초]
[B] 2초]
[C] 2초]
[C] 작업이 종료되었음
[A] 2초]
[B] 3초][A] 3초]
[A] 작업이 종료되었음

[B] 4초]
[B] 5초]
[B] 작업이 종료되었음
작업에 소요된 시간은 총 274초 입니다.


### 메일 발송에 필요한 변수 초기화


In [29]:
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "운영지원팀 <rnjswodyd1@gmail.com>"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다"

with open('mail/content.txt', 'r', encoding = 'utf - 8') as f:
    contentTmpl = f.read()

### 동기식 메일 발송


In [31]:
startTime = dt.datetime.now()

with open("mail/mail_list.csv", "r", encoding = "euc-kr") as f:
    csv = f.readlines()

    for line in csv:
        name, email, file1, file2 = line.strip().split(",")
        #print(name,email, file1, file2)

        toAddr = "{name} <{email}>".format(name=name, email=email)
        #print(toAddr)

        # 메일 제목
        subject = subjectTmpl.format(name=name, yy=year, mm=month)
        #print(subject)

        # 메일 내용
        content = contentTmpl.format(name=name, yy=year, mm=month, dd=day)
        #print(content)

        # 메일 보내기
        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endtime - startTime
print("작업에 소요된 시간은 총 %s초 입니다" % workTime.seconds)

document.pptx 가 첨부되었습니다
pay1.xlsx 가 첨부되었습니다
document.pptx 가 첨부되었습니다
pay2.xlsx 가 첨부되었습니다
document.pptx 가 첨부되었습니다
pay3.xlsx 가 첨부되었습니다
작업에 소요된 시간은 총 86020초 입니다


### 비동기식 메일 발송


In [35]:
# 동시 메일 발송 처리

startTime = dt.datetime.now()

with open("mail/mail_list.csv", "r", encoding = "euc-kr") as f:
    csv = f.readlines()

    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        for line in csv:
            name, email, file1, file2 = line.strip().split(",")
            #print(name,email, file1, file2)

            toAddr = "{name} <{email}>".format(name=name, email=email)
            #print(toAddr)

            # 메일 제목
            subject = subjectTmpl.format(name=name, yy=year, mm=month)
            #print(subject)

            # 메일 내용
            content = contentTmpl.format(name=name, yy=year, mm=month, dd=day)
            #print(content)

            # 메일 보내기
            # MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])
            executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

endTime = dt.datetime.now()
workTime = endTime - startTime
print("작업에 소요된 시간은 총 %s초 입니다" % workTime.seconds)

document.pptxdocument.pptx 가 첨부되었습니다
 가 첨부되었습니다
document.pptx 가 첨부되었습니다
pay2.xlsx 가 첨부되었습니다
pay1.xlsx 가 첨부되었습니다
pay3.xlsx 가 첨부되었습니다
작업에 소요된 시간은 총 4초 입니다
